In [37]:
!pip install keras-tuner

In [39]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [40]:
fashion_mnist=keras.datasets.fashion_mnist

In [41]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [42]:
train_images=train_images/255.0
test_images=test_images/255.0

In [43]:
train_images[0].shape

(28, 28)

In [44]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [49]:
def build_model(hp):
  model=keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
        kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
        kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
        activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [50]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [51]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist_Fashion')

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)


Search: Running Trial #6

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |64                |128               
conv_1_kernel     |5                 |3                 
conv_2_filter     |48                |48                
conv_2_kernel     |3                 |5                 
dense_1_units     |64                |80                
learning_rate     |0.001             |0.001             

Epoch 1/3
1688/1688 [==============================] - 143s 85ms/step - loss: 0.3990 - accuracy: 0.8549 - val_loss: 0.2924 - val_accuracy: 0.8910
Epoch 2/3
1688/1688 [==============================] - 143s 85ms/step - loss: 0.2646 - accuracy: 0.9037 - val_loss: 0.2580 - val_accuracy: 0.9073
Epoch 3/3
 790/1688 [=============>................] - ETA: 1:14 - loss: 0.2122 - accuracy: 0.9218

In [55]:
model=tuner_search.get_best_models(num_models=1)[0]

In [56]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        153648    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                1858640   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 2,014,378
Trainable params: 2,014,378
Non-trainable params: 0
_________________________________________________________________
